In [1]:
import os
import sqlite3
from app.helpers import parse_abs_data
from app.helpers import extract_citations
from app.helpers import insert_new_paper
from app.helpers import remove_paper

In [24]:
file_path = '/Users/ericweng/Git/CS6400-project/cit-HepTh-abstracts/2001/0101005.abs'
file_name = '0101005.abs'
with open(file_path, 'r', encoding='utf-8') as file:
    abs_data = file.read()
    parsed_data = parse_abs_data(abs_data)
    # Remove leading zeros from filename
    parsed_data['paper_id'] = str(int(file_name.replace('.abs', '')))

In [25]:
parsed_data

{'paper_id': '101005',
 'authors': 'Khaled Saaidi, Mohammad Khorrami',
 'title': 'Nonlocal two-dimensional Yang-Mills- and generalized Yang-Mills-theories',
 'comments': '13 pages, LaTeX 2.09',
 'subj_class': None,
 'journal_ref': 'Int.J.Mod.Phys. A15 (2000) 4749-4759',
 'abstract': 'A generalization of the two-dimensional Yang-Mills and generalized Yang-Mills\ntheory is introduced in which the building B-F theory is nonlocal in the\nauxiliary field. The classical and quantum properties of this nonlocal\ngeneralization are investigated and it is shown that for large gauge groups,\nthere exist a simple correspondence between the properties a nonlocal theory\nand its corresponding local theory.'}

In [26]:
# Connect to the existing database
connection = sqlite3.connect("papers.db")

In [35]:
# Create a cursor object
cursor = connection.cursor()
paper_id = '101005'
query = f"select * from papers where paper_id = {paper_id};"
# query = f"select * from citations where from_paper_id = {paper_id};"
# query = f"select * from co_citations where paper_1_id = {paper_id};"
results = cursor.execute(query)
for row in results:
    print(row)


('101005', 'Nonlocal two-dimensional Yang-Mills- and generalized Yang-Mills-theories', 'Khaled Saaidi, Mohammad Khorrami', '13 pages, LaTeX 2.09', None, 'Int.J.Mod.Phys. A15 (2000) 4749-4759', 'A generalization of the two-dimensional Yang-Mills and generalized Yang-Mills\ntheory is introduced in which the building B-F theory is nonlocal in the\nauxiliary field. The classical and quantum properties of this nonlocal\ngeneralization are investigated and it is shown that for large gauge groups,\nthere exist a simple correspondence between the properties a nonlocal theory\nand its corresponding local theory.', 3, '2000-12-30')


In [29]:
remove_paper(connection, '101005')

In [30]:
# Create a cursor object
cursor = connection.cursor()

paper_id = '101005'
# query = f"select * from papers where paper_id = {paper_id};"
# query = f"select * from citations where to_paper_id = {paper_id};"
query = f"select * from co_citations where paper_1_id = {paper_id};"

results = cursor.execute(query)
for row in results:
    print(row)


In [31]:
citations_from, citations_to = extract_citations(paper_id, '/Users/ericweng/GT/CS6400-project/cit-HepTh.txt')
print(citations_from, citations_to)

['9707081', '9407114', '9608144', '9612018', '9212090', '9204083', '9305047'] ['304068', '202170', '304204']


In [32]:
# :param conn: SQLite database connection.
# :param paper_metadata: Dictionary containing paper metadata (e.g., paper_id, title, authors, etc.).
# :param citations_from: List of papers that this paper cites.
# :param citations_to: List of papers that cite this paper.
# :param publication_date: Date when the paper was published.

insert_new_paper(connection, parsed_data, citations_from, citations_to, '2000-12-30')

In [13]:
connection.commit()
cursor.close()
connection.close()

ProgrammingError: Cannot operate on a closed database.